In [1]:
from __future__ import print_function
from csky.ipyconfig import *
from csky import bk, analysis, coord, dists, hyp, inj, llh, pdf, selections, trial, utils
import csky as cy
import histlite as hl
import healpy as hp
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
#from icecube import astro

mp_cpus = 15

In [2]:
ana_dir = utils.ensure_dir ('/data/user/drysewyk/csky_cache/ana_new')
repo = selections.mrichman_repo

In [3]:
ana7 = analysis.Analysis (repo, selections.PSDataSpecs.ps_7yr, dir=ana_dir)

Setting up Analysis for:
IC40, IC59, IC79, IC86_2011, IC86_2012_2014
Setting up IC40...
Reading /data/user/mrichman/data/analyses/ps_tracks/current/IC40_MC.npy ...
Reading /data/user/mrichman/data/analyses/ps_tracks/current/IC40_exp.npy ...
<- /data/user/drysewyk/csky_cache/ana_new/IC40.subanalysis.npy    
Setting up IC59...
Reading /data/user/mrichman/data/analyses/ps_tracks/current/IC59_MC.npy ...
Reading /data/user/mrichman/data/analyses/ps_tracks/current/IC59_exp.npy ...
<- /data/user/drysewyk/csky_cache/ana_new/IC59.subanalysis.npy    
Setting up IC79...
Reading /data/user/mrichman/data/analyses/ps_tracks/current/IC79_MC.npy ...
Reading /data/user/mrichman/data/analyses/ps_tracks/current/IC79_exp.npy ...
<- /data/user/drysewyk/csky_cache/ana_new/IC79.subanalysis.npy    
Setting up IC86_2011...
Reading /data/user/mrichman/data/analyses/ps_tracks/current/IC86_2011_MC.npy ...
Reading /data/user/mrichman/data/analyses/ps_tracks/current/IC86_2011_exp.npy ...
<- /data/user/drysewyk/csky

In [4]:
# def get_llh (a, src, sigsub):
#     # space PDF - use default background space PDF and signal acceptance parameterization
#     space_model = pdf.PointSourceSpacePDFRatioModel ( a, src, a.bg_space_param, a.acc_param, sigsub=sigsub )
#     # energy PDF - use default parameterization
#     energy_model = a.energy_pdf_ratio_model
#     # put it together
#     pdf_ratio_model = pdf.MultiPDFRatioModel (space_model, energy_model)
#     N = len (a.data)
#     llh_model = llh.LLHModel (pdf_ratio_model, N, sigsub=sigsub)
#     return llh_model

# def get_injs (a, llh_model, src, flux, cut_n_sigma, inj):
#     # note which event features we need
#     keep = llh_model.pdf_ratio_model.keep
#     # optimization: dec band cut
#     selector = cy.inj.DecBandSelector (src, cut_n_sigma=cut_n_sigma)
#     # truth: unscrambled data, for unblinding
#     truth = cy.inj.DataInjector (a, selector (a.data), keep, randomizers=[])
#     # bg: scrambled data
#     randomizers = [cy.inj.RARandomizer ()]#, cy.inj.DecRandomizer()]#, cy.inj.PoleRandomizer (np.radians (30))]
#     bg = cy.inj.DataInjector (a, selector (a.data), keep, randomizers=randomizers)
#     # sig: signal injection
#     sig = cy.inj.PointSourceInjector (a, src, flux, keep) if inj else None
#     return truth, bg, sig

# def get_tr (src, ana=ana7, flux=hyp.PowerLawFlux (2), inj=True, use_energy=True, cut_n_sigma=5, sigsub=False, _fit_null=True, mp_cpus=mp_cpus):
#     # get_llh needs the src list
#     llh_kw = dict (src=src, sigsub=sigsub)
#     # get_injs needs the src list and signal spectrum
#     inj_kw = dict (src=src, flux=flux, cut_n_sigma=cut_n_sigma, inj=inj)
#     # trial.get_trial_runner loops over sub analyses and gives a single TrialRunner
#     return trial.get_trial_runner (
#         ana, get_llh, get_injs, llh_kw=llh_kw, inj_kw=inj_kw, mp_cpus=mp_cpus)


# def get_getters (ana=ana7, extension=0, cut_n_sigma=3, sigsub=False):
#     def get_tr_skymap (src=None, cut_n_sigma=3, _fit_null=True, inj=True):
#         if src is None:
#             src = utils.Sources (ra=0, dec=0, extension=extension)
#         src['extension'] = extension * np.ones_like (src.dec)
#         return get_tr (src, ana=ana, cut_n_sigma=cut_n_sigma, sigsub=sigsub, _fit_null=_fit_null, inj=inj)
#     def get_selector (dec):
#         return inj.DecBandSelector (utils.Sources (dec=dec, extension=extension))
#     return get_tr_skymap, get_selector

# def plot_result (v, clabel, title='', vmin=None, vmax=None, cmap='viridis', projection='aitoff'):
#     if vmin is None:
#         vmin = 0
#     if vmax is None:
#         vmax = np.ceil (v.max())
#     fig, ax = plt.subplots (figsize=(8,6), subplot_kw=dict (projection=projection))
#     sp = csp.SkyPlotter (pc_kw=dict (cmap=cmap, vmin=vmin, vmax=vmax))
#     v = hp.ud_grade (v, 256)
#     mesh, cb = sp.plot_map (ax, v, n_ticks=2, titleticks=bool (title))
#     sp.plot_gp (ax, color='.5', lw=.5, alpha=.25)
#     ax.grid (alpha=.5, ls='-')
#     cb.set_label (clabel)
#     ax.set_title (title)
#     plt.tight_layout()
#     return fig, ax

In [11]:
def get_llh (a, src, cut_n_sigma, sigsub):
    # space PDF - use default background space PDF and signal acceptance parameterization
    space_model = pdf.PointSourceSpacePDFRatioModel ( a, src, a.bg_space_param, a.acc_param, cut_n_sigma=cut_n_sigma, sigsub=sigsub )
    # energy PDF - use default parameterization
    energy_model = a.energy_pdf_ratio_model
    # put it together
    pdf_ratio_model = pdf.MultiPDFRatioModel (space_model, energy_model)
    N = len (a.data)
    llh_model = llh.LLHModel (pdf_ratio_model, N, cut_n_sigma=cut_n_sigma, sigsub=sigsub)
    return llh_model

def get_injs (a, llh_model, src, llh_src, flux, cut_n_sigma, inj):
    # note which event features we need
    keep = llh_model.pdf_ratio_model.keep
    # optimization: dec band cut
    selector = cy.inj.DecBandSelector (llh_src, cut_n_sigma=cut_n_sigma)
    # truth: unscrambled data, for unblinding
    truth = cy.inj.DataInjector (a, selector (a.data), keep, randomizers=[])
    # bg: scrambled data
    randomizers = [cy.inj.RARandomizer ()]#, cy.inj.DecRandomizer()]#, cy.inj.PoleRandomizer (np.radians (30))]
    bg = cy.inj.DataInjector (a, selector (a.data), keep, randomizers=randomizers)
    # sig: signal injection
    sig = cy.inj.PointSourceInjector (a, src, flux, keep) if inj else None
    return truth, bg, sig

def get_tr (src, ana=ana7, ext=0, llh_ext=None, flux=hyp.PowerLawFlux (2), inj=True, use_energy=True, cut_n_sigma=None, sigsub=False, _fit_null=True, mp_cpus=mp_cpus):
    llh_ext = ext if llh_ext is None else llh_ext
    llh_src = cy.utils.Sources(ra=src.ra, dec=src.dec, extension=llh_ext)
    
    if cut_n_sigma is None:
        cut_n_sigma = 5 if llh_ext < np.radians(3) else 3
    
    # get_llh needs the src list
    llh_kw = dict (src=llh_src, cut_n_sigma=cut_n_sigma, sigsub=sigsub)
    # get_injs needs the src list and signal spectrum
    inj_kw = dict (src=src, llh_src=llh_src, flux=flux, cut_n_sigma=cut_n_sigma, inj=inj)
    # trial.get_trial_runner loops over sub analyses and gives a single TrialRunner
    return trial.get_trial_runner (
        ana, get_llh, get_injs, llh_kw=llh_kw, inj_kw=inj_kw, mp_cpus=mp_cpus)

def get_sens(ana, ra, sindec, ext, llh_ext=None, cut_n_sigma=None, sigsub=False, gamma=2, batch_size=500):
    src = cy.utils.Sources(ra=0, dec=np.arcsin(sindec),extension=np.radians(ext))
    flux = hyp.PowerLawFlux(gamma)
    tr = get_tr(src=src, ana=ana, ext=ext, llh_ext=llh_ext, flux=flux, cut_n_sigma=cut_n_sigma, sigsub=sigsub)
    bg = cy.dists.Chi2TSD(tr.get_many_fits(500))
    sens = tr.find_n_sig(bg.median(),0.9,batch_size=batch_size,max_batch_size=1000,tol=0.03, coverage=2) 
    sens['flux'] = tr.to_E2dNdE(sens['n_sig'], E0=100, unit=1e3)
    return sens['flux']

def get_disc(ana, ra, sindec, ext, llh_ext=None, cut_n_sigma=None, sigsub=False, gamma=2, batch_size=500):
    src = cy.utils.Sources(ra=0, dec=np.arcsin(sindec),extension=np.radians(ext))
    flux = hyp.PowerLawFlux(gamma)
    tr = get_tr(src=src, ana=ana, ext=ext, llh_ext=llh_ext, flux=flux, cut_n_sigma=cut_n_sigma, sigsub=sigsub)
    bg = cy.dists.Chi2TSD(tr.get_many_fits(500))
    sens = tr.find_n_sig(bg.isf_nsigma(5,fit=True),0.5,batch_size=batch_size,max_batch_size=1000,tol=0.03, coverage=2) 
    sens['flux'] = tr.to_E2dNdE(sens['n_sig'], E0=100, unit=1e3)
    return sens['flux']

In [12]:
# def get_sens(ana,ra,sindec,ext,gamma,batch_size=500):
#     src = cy.utils.Sources(ra=0, dec=np.arcsin(sindec),extension=np.radians(ext))
#     #flux = hyp.PowerLawFlux(gamma)
#     tr = get_tr(src, ana)#, flux=flux)
#     bg = cy.dists.Chi2TSD(tr.get_many_fits(500))
#     sens = tr.find_n_sig(bg.median(),0.9,batch_size=batch_size,max_batch_size=1000,tol=0.03) 
#     sens['flux'] = tr.to_E2dNdE(sens['n_sig'], E0=100, unit=1e3)
#     return sens['flux']

# def get_sens_sigsub(ana,ra,sindec,ext,gamma,batch_size=500):
#     src = cy.utils.Sources(ra=0, dec=np.arcsin(sindec),extension=np.radians(ext))
#     #flux = hyp.PowerLawFlux(gamma)
#     tr = get_tr(src, ana, sigsub=True)#, flux=flux)
#     bg = cy.dists.Chi2TSD(tr.get_many_fits(500))
#     sens = tr.find_n_sig(bg.median(),0.9,batch_size=batch_size,max_batch_size=1000,tol=0.03) 
#     sens['flux'] = tr.to_E2dNdE(sens['n_sig'], E0=100, unit=1e3)
#     return sens['flux']

# def get_discovery(ana,ra,sindec,ext,gamma,batch_size=500):
#     src = utils.Sources(ra=0, dec=np.arcsin(sindec),extension=np.radians(ext))
#     #flux = hyp.PowerLawFlux(gamma)
#     tr = get_tr(src, ana)#, flux=flux)
#     bg = cy.dists.Chi2TSD(tr.get_many_fits(500))
#     sens = tr.find_n_sig(bg.isf_nsigma(5,fit=True),0.5,batch_size=batch_size,max_batch_size=1000,tol=0.03) 
#     sens['flux'] = tr.to_E2dNdE(sens['n_sig'], E0=100, unit=1e3)
#     return sens['flux']

# def get_discovery_sigsub(ana,ra,sindec,ext,gamma,batch_size=500):
#     src = utils.Sources(ra=0, dec=np.arcsin(sindec),extension=np.radians(ext))
#     #flux = hyp.PowerLawFlux(gamma)
#     tr = get_tr(src, ana, sigsub=True)#, flux=flux)
#     bg = cy.dists.Chi2TSD(tr.get_many_fits(500))
#     sens = tr.find_n_sig(bg.isf_nsigma(5,fit=True),0.5,batch_size=batch_size,max_batch_size=1000,tol=0.03) 
#     sens['flux'] = tr.to_E2dNdE(sens['n_sig'], E0=100, unit=1e3)
#     return sens['flux']

In [10]:
sindec_array = np.linspace(-0.9,0.9,1)
%time sens_0_test = [get_sens(ana7,ra=0,sindec=d,ext=0,llh_ext=None, cut_n_sigma=None, gamma=2, sigsub=True) for d in sindec_array]
#np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/Sensitivity/sindec_array.npy',dec_array)
#np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/DiscoveryPotential/sindec_array.npy',dec_array)

Performing 500 background trials using 15 cores:
        500/500 trials complete.   
Start time: 2019-02-21 15:05:16.203128
Using 15 cores.
* Starting initial scan for 90% of 50 trials with TS >= 0.103...
  n_sig = 3.000 ... frac = 0.86000
  n_sig = 6.000 ... frac = 0.98000
* Generating batches of 500 trials...
n_trials | n_inj    0.00    2.40    4.80    7.20    9.60   12.00  | n_sig (relative error)
500      |         51.4%   81.8%   96.0%   98.4%   99.8%   99.8%  |      3.478 (+/-  4.1%) [spline]
1000     |         49.2%   81.5%   96.3%   98.7%   99.9%   99.9%  |      3.466 (+/-  3.1%) [spline]
End time: 2019-02-21 15:06:15.276354
Elapsed time: 0:00:59.073226
CPU times: user 1.12 s, sys: 8.07 s, total: 9.19 s
Wall time: 1min 4s


In [8]:
np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/Sensitivity/sens_0_test.npy',sens_0_test)


In [14]:
%time sens_1 = [get_sens(ana7,ra=0,sindec=d,ext=1,gamma=2) for d in dec_array]
np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/Sensitivity/sens_1.npy',sens_1)
%time disc_1 = [get_discovery(ana7,ra=0,sindec=d,ext=1,gamma=2) for d in dec_array]
np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/DiscoveryPotential/disc_1.npy',disc_1)

Performing 500 background trials using 15 cores:
        500/500 trials complete.   
Start time: 2019-02-14 07:52:16.645193
Using 15 cores.
* Starting initial scan for 90% of 50 trials with TS >= 0.279...
  n_sig = 3.000 ... frac = 0.66000
  n_sig = 6.000 ... frac = 0.88000
  n_sig = 9.000 ... frac = 0.96000
* Generating batches of 500 trials...
n_trials | n_inj    0.00    2.70    5.40    8.10   10.80   13.50  | n_sig (relative error)
500      |         50.4%   74.4%   85.2%   93.6%   97.0%   98.8%  |      6.757 (+/-  4.4%) [spline]
1000     |         49.3%   74.0%   85.1%   94.2%   97.3%   97.9%  |      6.664 (+/-  2.9%) [spline]
End time: 2019-02-14 07:53:03.184730
Elapsed time: 0:00:46.539537
Performing 500 background trials using 15 cores:
        500/500 trials complete.   
Start time: 2019-02-14 07:53:07.618127
Using 15 cores.
* Starting initial scan for 90% of 50 trials with TS >= 0.012...
  n_sig = 3.000 ... frac = 0.86000
  n_sig = 6.000 ... frac = 0.86000
  n_sig = 9.000 ... 

  n_sig = 54.000 ... frac = 0.46000
  n_sig = 57.000 ... frac = 0.46000
  n_sig = 60.000 ... frac = 0.60000
* Generating batches of 500 trials...
n_trials | n_inj    0.00   18.00   36.00   54.00   72.00   90.00  | n_sig (relative error)
500      |          0.0%    0.0%    6.0%   37.2%   79.2%   96.2%  |     59.396 (+/-  1.2%) [spline]
End time: 2019-02-14 08:11:27.224289
Elapsed time: 0:00:56.201478
Performing 500 background trials using 15 cores:
        500/500 trials complete.   
Start time: 2019-02-14 08:11:32.845325
Using 15 cores.
* Starting initial scan for 50% of 50 trials with TS >= 43.355...
  n_sig = 3.000 ... frac = 0.00000
  n_sig = 6.000 ... frac = 0.00000
  n_sig = 9.000 ... frac = 0.00000
  n_sig = 12.000 ... frac = 0.00000
  n_sig = 15.000 ... frac = 0.00000
  n_sig = 18.000 ... frac = 0.00000
  n_sig = 21.000 ... frac = 0.00000
  n_sig = 24.000 ... frac = 0.00000
  n_sig = 27.000 ... frac = 0.00000
  n_sig = 30.000 ... frac = 0.02000
  n_sig = 33.000 ... frac = 0.0200

In [15]:
%time sens_2 = [get_sens(ana7,ra=0,sindec=d,ext=2,gamma=2) for d in dec_array]
np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/Sensitivity/sens_2.npy',sens_2)
%time disc_2 = [get_discovery(ana7,ra=0,sindec=d,ext=2,gamma=2) for d in dec_array]
np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/DiscoveryPotential/disc_2.npy',disc_2)

Performing 500 background trials using 15 cores:
        500/500 trials complete.   
Start time: 2019-02-14 08:14:05.949265
Using 15 cores.
* Starting initial scan for 90% of 50 trials with TS >= 0.429...
  n_sig = 3.000 ... frac = 0.56000
  n_sig = 6.000 ... frac = 0.84000
  n_sig = 9.000 ... frac = 0.80000
  n_sig = 12.000 ... frac = 0.88000
  n_sig = 15.000 ... frac = 0.96000
* Generating batches of 500 trials...
n_trials | n_inj    0.00    4.50    9.00   13.50   18.00   22.50  | n_sig (relative error)
500      |         53.8%   74.2%   88.6%   95.2%   97.8%   99.0%  |      9.723 (+/-  6.1%) [spline]
1000     |         49.0%   72.2%   87.5%   94.5%   97.3%   99.4%  |     10.256 (+/-  4.0%) [spline]
End time: 2019-02-14 08:15:15.411862
Elapsed time: 0:01:09.462597
Performing 500 background trials using 15 cores:
        500/500 trials complete.   
Start time: 2019-02-14 08:15:22.186164
Using 15 cores.
* Starting initial scan for 90% of 50 trials with TS >= 0.011...
  n_sig = 3.000 ..

Performing 500 background trials using 15 cores:
        500/500 trials complete.   
Start time: 2019-02-14 08:30:08.092948
Using 15 cores.
* Starting initial scan for 50% of 50 trials with TS >= 50.126...
  n_sig = 3.000 ... frac = 0.00000
  n_sig = 6.000 ... frac = 0.00000
  n_sig = 9.000 ... frac = 0.00000
  n_sig = 12.000 ... frac = 0.00000
  n_sig = 15.000 ... frac = 0.00000
  n_sig = 18.000 ... frac = 0.00000
  n_sig = 21.000 ... frac = 0.00000
  n_sig = 24.000 ... frac = 0.00000
  n_sig = 27.000 ... frac = 0.00000
  n_sig = 30.000 ... frac = 0.00000
  n_sig = 33.000 ... frac = 0.04000
  n_sig = 36.000 ... frac = 0.12000
  n_sig = 39.000 ... frac = 0.12000
  n_sig = 42.000 ... frac = 0.26000
  n_sig = 45.000 ... frac = 0.20000
  n_sig = 48.000 ... frac = 0.18000
  n_sig = 51.000 ... frac = 0.32000
  n_sig = 54.000 ... frac = 0.48000
  n_sig = 57.000 ... frac = 0.46000
  n_sig = 60.000 ... frac = 0.66000
* Generating batches of 500 trials...
n_trials | n_inj    0.00   18.00   36.0

  n_sig = 66.000 ... frac = 0.06000
  n_sig = 69.000 ... frac = 0.08000
  n_sig = 72.000 ... frac = 0.16000
  n_sig = 75.000 ... frac = 0.24000
  n_sig = 78.000 ... frac = 0.38000
  n_sig = 81.000 ... frac = 0.22000
  n_sig = 84.000 ... frac = 0.36000
  n_sig = 87.000 ... frac = 0.40000
  n_sig = 90.000 ... frac = 0.34000
  n_sig = 93.000 ... frac = 0.44000
  n_sig = 96.000 ... frac = 0.54000
* Generating batches of 500 trials...
n_trials | n_inj    0.00   28.80   57.60   86.40  115.20  144.00  | n_sig (relative error)
500      |          0.0%    0.0%    7.8%   48.4%   86.0%   99.8%  |     87.564 (+/-  1.4%) [spline]
End time: 2019-02-14 08:42:09.194139
Elapsed time: 0:01:45.743780
Performing 500 background trials using 15 cores:
        500/500 trials complete.   
Start time: 2019-02-14 08:42:17.019514
Using 15 cores.
* Starting initial scan for 50% of 50 trials with TS >= 40.754...
  n_sig = 3.000 ... frac = 0.00000
  n_sig = 6.000 ... frac = 0.00000
  n_sig = 9.000 ... frac = 0.0000

In [17]:
#%time sens_3 = [get_sens(ana7,ra=0,sindec=d,ext=3,gamma=2) for d in dec_array]
#np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/Sensitivity/sens_3.npy',sens_3)
%time disc_3 = [get_discovery(ana7,ra=0,sindec=d,ext=3,gamma=2) for d in dec_array]
np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/DiscoveryPotential/disc_3.npy',disc_3)


Performing 500 background trials using 15 cores:
        500/500 trials complete.   
Start time: 2019-02-14 09:41:06.971045
Using 15 cores.
* Starting initial scan for 50% of 50 trials with TS >= 52.208...
  n_sig = 3.000 ... frac = 0.00000
  n_sig = 6.000 ... frac = 0.00000
  n_sig = 9.000 ... frac = 0.00000
  n_sig = 12.000 ... frac = 0.00000
  n_sig = 15.000 ... frac = 0.00000
  n_sig = 18.000 ... frac = 0.00000
  n_sig = 21.000 ... frac = 0.00000
  n_sig = 24.000 ... frac = 0.00000
  n_sig = 27.000 ... frac = 0.00000
  n_sig = 30.000 ... frac = 0.00000
  n_sig = 33.000 ... frac = 0.00000
  n_sig = 36.000 ... frac = 0.00000
  n_sig = 39.000 ... frac = 0.06000
  n_sig = 42.000 ... frac = 0.00000
  n_sig = 45.000 ... frac = 0.10000
  n_sig = 48.000 ... frac = 0.12000
  n_sig = 51.000 ... frac = 0.14000
  n_sig = 54.000 ... frac = 0.36000
  n_sig = 57.000 ... frac = 0.38000
  n_sig = 60.000 ... frac = 0.30000
  n_sig = 63.000 ... frac = 0.44000
  n_sig = 66.000 ... frac = 0.52000
* Gen

OSError: [Errno 12] Cannot allocate memory

In [ ]:
%time sens_4 = [get_sens(ana7,ra=0,sindec=d,ext=4,gamma=2) for d in dec_array]
np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/Sensitivity/sens_4.npy',sens_4)
%time disc_4 = [get_discovery(ana7,ra=0,sindec=d,ext=4,gamma=2) for d in dec_array]
np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/DiscoveryPotential/disc_4.npy',disc_4)

In [ ]:
%time sens_5 = [get_sens(ana7,ra=0,sindec=d,ext=5,gamma=2) for d in dec_array]
np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/Sensitivity/sens_5.npy',sens_5)
%time disc_5 = [get_discovery(ana7,ra=0,sindec=d,ext=5,gamma=2) for d in dec_array]
np.save('/data/user/drysewyk/llh_sandbox/NumPyArrays/DiscoveryPotential/disc_5.npy',disc_5)